# Módulo 4 - Atividade Prática - Primeira aula interativa


In [1]:
import pandas as pd
import numpy as pnp
import zipfile
import requests
from io import BytesIO
import os

In [5]:
# truque que funciona apenas om o jupyter notebook
#%%time
# vscode ja tem time


#criar diretorio
os.makedirs("./enem2019b", exist_ok=True)

#Definir url
url = "https://download.inep.gov.br/microdados/microdados_enem_2019.zip"

# Download do conteúdo da url
filebytes = BytesIO(
    requests.get(url, stream=True).content
)
#stream == truque


# Descompactação do zip
myzip = zipfile.ZipFile(filebytes)

# Pasta destino da descompactação
myzip.extractall("./enem2019b")



## Leitura dos Dados



In [7]:
#pulo do gato pra ir mais rapido

import dask.dataframe as dd

In [18]:
enemB = dd.read_csv("./enem2019b/DADOS/MICRODADOS_ENEM_2019.csv",
                    sep=";",
                    decimal=".",
                    encoding='latin-1',
                    dtype={'NU_IDADE': 'float64'}

                    )

In [19]:
enemB.head(10)

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001595656,2019,3552205,Sorocaba,35,SP,36.0,M,1,3,...,A,A,A,A,A,A,C,A,C,B
1,190001421546,2019,2910800,Feira de Santana,29,BA,23.0,M,1,1,...,A,A,A,B,A,A,E,A,A,B
2,190001133210,2019,2304400,Fortaleza,23,CE,39.0,F,1,3,...,A,A,A,B,A,A,C,A,A,B
3,190001199383,2019,1721000,Palmas,17,TO,25.0,F,1,1,...,A,A,A,C,A,B,D,A,B,B
4,190001237802,2019,3118601,Contagem,31,MG,22.0,F,1,1,...,B,A,A,B,A,A,C,B,B,B
5,190001782198,2019,5107602,Rondonópolis,51,MT,37.0,M,2,2,...,B,A,A,B,A,A,E,A,B,B
6,190001421548,2019,2924009,Paulo Afonso,29,BA,22.0,F,1,3,...,A,A,A,B,A,A,B,A,A,A
7,190001595657,2019,3550308,São Paulo,35,SP,20.0,M,1,3,...,B,A,A,B,A,A,C,A,A,B
8,190001592264,2019,3550308,São Paulo,35,SP,20.0,F,1,1,...,B,A,A,B,A,A,B,B,A,A
9,190001592266,2019,3550308,São Paulo,35,SP,17.0,M,1,1,...,B,A,A,B,A,B,C,B,B,B


In [20]:
enemMG = enemB.loc[enemB.SG_UF_RESIDENCIA == "MG"]

In [21]:
enemMG.to_csv("./enem2019b/enemAula.csv", index=False)

['d:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\00.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\01.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\02.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\03.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\04.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\05.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\06.part',
 'd:/ONEDRIVE/EDUCACAO/IGTI/BOOTCAMP ENGENHARIA DE DADOS/GitHub/modulo-4/Trabalho Prático/enem2019b/enemAula.csv\\07.part',
 'd:/ONE

In [23]:
enemMG.groupby("TP_SEXO").agg({
    "NU_IDADE": "mean"
}).compute()

,NU_IDADE
TP_SEXO,
F,21.872028
M,22.258798


In [27]:
enemMG.groupby(["TP_SEXO", 'NO_MUNICIPIO_RESIDENCIA']).agg({
    "NU_IDADE": "mean",
    "NU_NOTA_MT": "mean"
}).compute()

NU_IDADE  NU_NOTA_MT
TP_SEXO NO_MUNICIPIO_RESIDENCIA                       
F       Abadia dos Dourados      19.746988  506.700000
        Abaeté                   20.910299  539.117411
        Abre Campo               19.689119  536.921935
        Acaiaca                  22.571429  474.465116
        Aguanil                  18.357143  532.605714
...                                    ...         ...
M       São José da Safira       22.571429  491.790000
        Taparuba                 19.038462  566.055000
        Seritinga                20.500000  589.266667
        Serranos                 20.647059  455.250000
        Água Comprida            25.666667  518.800000

[1706 rows x 2 columns]